# I. Install Detectron2

In [ ]:
!python -m pip install pyyaml==5.1
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

## 1.1 Check Detectron2 Version

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.1 ; cuda:  cu118
detectron2: 0.6


# 2. Setup: Import Library

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

In this section, we show how to train an existing detectron2 model on a custom dataset in COCO format.

## 2.1 Prepare the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/colab_dataset/image_train_dark.zip"

Register the custom dataset to Detectron2, following the detectron2 custom dataset tutorial. Here, the dataset is in COCO format, therefore we register into Detectron2's standard format. User should write such a function when using a dataset in custom format. See the tutorial for more details.

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import get_detection_dataset_dicts
from detectron2.data.datasets import builtin_meta

In [ ]:
for d in ["train", "val"]:
    # register_coco_instances(f"orchid_{d}", {}, f"/content/leaves_core/annotations/{d}.json", f"/content/leaves_core/{d}")
    register_coco_instances(f"dark_{d}", {}, f"/content/dark/annotations/{d}.json", f"/content/dark/{d}")
MetadataCatalog.get("dark_train").set(thing_classes = ['bud','core']).set(thing_dataset_id_to_contiguous_id={1: 0,2: 1})
MetadataCatalog.get("dark_val").set(thing_classes = ['bud','core']).set(thing_dataset_id_to_contiguous_id={1: 0,2: 1})

To verify the data loading is correct, let's visualize the annotations of a randomly selected sample in the training set:

In [ ]:
from detectron2.utils.visualizer import  Visualizer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import ColorMode
import os, json, cv2, random
import matplotlib.pyplot as plt

In [ ]:
def plot_sample(dataset_name,n=1):
  dataset_custom = DatasetCatalog.get(dataset_name)
  dataset_custom_mega = MetadataCatalog.get(dataset_name)
  for s in random.sample(dataset_custom,n):
    im = cv2.imread(s["file_name"])
    v = Visualizer(im[:,:,::-1],metadata=dataset_custom_mega, scale = 0.5)
    v = v.draw_dataset_dict(s)
    plt.figure(figsize=(15,20))
    plt.imshow(v.get_image())
    plt.show()

In [ ]:
plot_sample(dataset_name="dark_val",n=1)

## 3. Training

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader
import detectron2.utils.comm as comm
import os

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dark_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2 #@param
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.3
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH =  8 #@param
cfg.SOLVER.BASE_LR = 0.0025 #@param # pick a good LR
cfg.SOLVER.MAX_ITER = 1000 #@param    # 300 iterations seems good enough for 100 frames dataset; you will need to train longer for a practical dataset
cfg.SOLVER.CHECKPOINT_PERIOD = 1000 #@param
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 #@param   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #@param  #  (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
trainer.train()

## 4. Inference & evaluation using the trained model
Now, let's run inference with the trained model on the validation dataset. First, let's create a predictor using the model we just trained:

### 4.1 Inference

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously.
# We simply update the weights with the newly trained ones to perform inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
# set a custom testing threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15   #@param {type: "slider", min:0.0, max:1.0, step: 0.01}
predictor = DefaultPredictor(cfg)

In [ ]:
import glob
import random
from detectron2.utils.visualizer import Visualizer

In [ ]:
image_path = glob.glob("/content/dark/val/*.jpg")

In [ ]:
def on_image(image_path,predictor):
  dataset_custom_mega = MetadataCatalog.get("dark_train")
  im = cv2.imread(image_path)
  outputs = predictor(im)
  # v = Visualizer(im[:,:,::-1], metadata = {},scale = 0.5, instance_mode = ColorMode.SEGMENTATION)
  v = Visualizer(im[:,:,::-1], metadata = dataset_custom_mega,scale = 0.5,)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  predicted = outputs["instances"].pred_classes.to("cpu")
  print("classes",predicted)
  plt.figure(figsize = (14,10))
  plt.imshow(v.get_image())
  plt.show()

In [ ]:
for image in random.sample(image_path,10):
  im = cv2.imread(image)
  cv2_imshow(im)
  on_image(image,predictor)

## 4.2 Evaluation: AP (Average Precision)

A more robust way to evaluate the model is to use a metric called Average Precision (AP) already implemented in the detectron2 package. If you want more precision on what the AP is, you can take a look [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) and [here](https://en.wikipedia.org/w/index.php?title=Information_retrieval&oldid=793358396#Average_precision).

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [ ]:
evaluator = COCOEvaluator("dark_valid", cfg, False, output_dir="/content/eval_output/")
val_loader = build_detection_test_loader(cfg, "dark_valid")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`